In [ ]:
from bottle import Bottle, route, run, template, get, post, debug, static_file, request, redirect, response
import time
import random
import string
import logging
import logging.handlers

log = logging.getLogger('bottle')
log.setLevel('INFO')
h = logging.handlers.TimedRotatingFileHandler(
    'logs/nlog', when='midnight', backupCount=9999)
f = logging.Formatter('%(asctime)s %(levelname)-8s %(message)s')
h.setFormatter(f)
log.addHandler(h)


secretKey = "vcxv&TTFS98v334sd123fsdhkldasdjghd6DFOUFYWE&FY"


from users import users
from statics import messDict

app = Bottle()


@route('/static/<path:path>', name='static')
def static(path):
    return static_file(path, root='./static')


def checkAuth():
    loginName = request.get_cookie("user", secret=secretKey)
    randStr = request.get_cookie("randStr", secret=secretKey)
    log.info(str(loginName) + ' ' + request.method + ' ' +
             request.url + ' ' + request.environ.get('REMOTE_ADDR'))
    if (loginName in users) and \
        (users[loginName].get("randStr", "") == randStr) and \
        (users[loginName]["loggedIn"] == True) and \
        (time.time() - users[loginName]["lastSeen"] < 3600):
        users[loginName]["lastSeen"] = time.time()
        return loginName
    return redirect('/login')


@route('/login')
@route('/login/')
@route('/login', method='POST')
def login():
    loginName = request.forms.getunicode('inputLogin', default=False)
    password = request.forms.getunicode('inputPassword', default=False)
    randStr = ''.join(random.choice(
        string.ascii_uppercase + string.digits) for _ in range(18))
    
    log.info(str(loginName) + ' ' + request.method + ' ' +
             request.url + ' ' + request.environ.get('REMOTE_ADDR'))
    
    if (loginName in users) and users[loginName]["password"] == password:
        response.set_cookie("user", loginName, secret=secretKey)
        response.set_cookie("randStr", randStr, secret=secretKey)
        users[loginName]["loggedIn"] = True
        users[loginName]["randStr"] = randStr
        users[loginName]["lastSeen"] = time.time()
        
        redirect('/index')
        return True
    else:
        return template('login')
    return template('login')


@route('/')
@route('/index')
@route('/index/')
@route('/index/<message>')
def index(message=''):
    loginName = checkAuth()
    return template('index', message=messDict.get(message, ""), loginName=loginName)

@route('/login/')
@route('/login')
def login():
    return template("login")

@route('/register')
@route('/register/')
def regiser():
    return template("register")

@route('/one_sample_ttest')
@route('/one_sample_ttest/')
def one_sample_ttest():
    return template("one_sample_ttest")

@route('/static/<path:path>', name='static')
def static(path):
    return static_file(path, root='./static')

# Debug = True - changes in template are visible immidietly
run(host='localhost', port=8012, debug=True)

Bottle v0.12.19 server starting up (using WSGIRefServer())...
Listening on http://localhost:8012/
Hit Ctrl-C to quit.

127.0.0.1 - - [13/Jan/2022 10:41:02] "GET / HTTP/1.1" 303 0
127.0.0.1 - - [13/Jan/2022 10:41:03] "GET /login HTTP/1.1" 200 3113
127.0.0.1 - - [13/Jan/2022 10:41:03] "GET /static/js/scripts.js HTTP/1.1" 304 0
127.0.0.1 - - [13/Jan/2022 10:41:03] "GET /static/css/styles.css HTTP/1.1" 304 0
127.0.0.1 - - [13/Jan/2022 10:41:03] "GET /favicon.ico HTTP/1.1" 404 742
127.0.0.1 - - [13/Jan/2022 10:41:17] "POST /login HTTP/1.1" 303 0
127.0.0.1 - - [13/Jan/2022 10:41:17] "GET /index HTTP/1.1" 200 4947
127.0.0.1 - - [13/Jan/2022 10:41:17] "GET /static/img1.jpg HTTP/1.1" 304 0
127.0.0.1 - - [13/Jan/2022 10:42:57] "GET /about HTTP/1.1" 404 730
127.0.0.1 - - [13/Jan/2022 10:43:14] "GET / HTTP/1.1" 200 4947
127.0.0.1 - - [13/Jan/2022 10:43:21] "GET /login HTTP/1.1" 200 3113
127.0.0.1 - - [13/Jan/2022 10:46:34] "GET /login HTTP/1.1" 200 3157
127.0.0.1 - - [13/Jan/2022 10:46:34] "GET /i

127.0.0.1 - - [13/Jan/2022 11:56:09] "GET /index HTTP/1.1" 200 6013
127.0.0.1 - - [13/Jan/2022 11:56:15] "GET /index HTTP/1.1" 200 6013
127.0.0.1 - - [13/Jan/2022 11:56:21] "GET /index HTTP/1.1" 200 6013
127.0.0.1 - - [13/Jan/2022 11:56:53] "GET /index HTTP/1.1" 200 6014
127.0.0.1 - - [13/Jan/2022 11:57:11] "GET /index HTTP/1.1" 200 6051
127.0.0.1 - - [13/Jan/2022 11:57:21] "GET /index HTTP/1.1" 200 6052
127.0.0.1 - - [13/Jan/2022 11:57:34] "GET /index HTTP/1.1" 200 6037
127.0.0.1 - - [13/Jan/2022 11:57:40] "GET /index HTTP/1.1" 200 6052
127.0.0.1 - - [13/Jan/2022 11:58:55] "GET /login HTTP/1.1" 200 3308
127.0.0.1 - - [13/Jan/2022 11:59:05] "GET /register HTTP/1.1" 200 5188
127.0.0.1 - - [13/Jan/2022 11:59:08] "GET /login HTTP/1.1" 200 3308
127.0.0.1 - - [13/Jan/2022 11:59:10] "POST /login HTTP/1.1" 303 0
127.0.0.1 - - [13/Jan/2022 11:59:10] "GET /index HTTP/1.1" 200 6052
127.0.0.1 - - [13/Jan/2022 12:02:57] "GET /login HTTP/1.1" 200 3308
127.0.0.1 - - [13/Jan/2022 12:03:51] "GET /logi